In [1]:
import pandas as pd
import numpy as np
import goldsberry
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel
import numpy as np

Seasons = ['2012-13', '2013-14', '2014-15', '2015-16']
Game_Data = {}
gameids = goldsberry.GameIDs()
for i in range(len(Seasons)):
    gameids.get_new_data(Season=Seasons[i])
    Game_Data[Seasons[i]] = pd.DataFrame(gameids.game_list())

team_ids =  list(Game_Data[Seasons[0]]['TEAM_ID'].value_counts().index)
WL_dict = {'W':1, 'L': -1}


def stats_per_season(season_index):
    goldsberry.apiparams.p_team_season['Season'] = Seasons[season_index]
    data = [goldsberry.team.season_stats(team).overall()[0] for team in team_ids]
    season_data = pd.DataFrame(data)
    season_data = season_data.set_index(['TEAM_ID'])
    return season_data


Stats_Data = {}
for i in range(len(Seasons)):
    Stats_Data[Seasons[i]] = stats_per_season(i)


/Users/jacobperricone/anaconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [11]:

def create_X(season_index):
    df = Game_Data[Seasons[season_index]]
    unique_game_ids = list(df['GAME_ID'].value_counts().index)
    if season_index != 0:
        prev_season = season_index - 1
        avg_season_data = Stats_Data[Seasons[prev_season]]
    else:
        print("We Need Prev Data For Stats")
        return

    X = pd.DataFrame()
    Y = []
    for i,game in enumerate(unique_game_ids):
        print "Number %s out of %s" % (i, len(unique_game_ids))
        rows = df[['WL', 'TEAM_ID']].ix[df['GAME_ID'] == game]
        stat_columns =[x for x in list(avg_season_data.columns) if 'TEAM_NAME' not in x and 'GROUP_SET' not in x and 'GROUP_VALUE' not in x]
        tmp = goldsberry.game.boxscore_summary(game).game_summary()[0]
        home_team_data = avg_season_data.ix[tmp['HOME_TEAM_ID'] ][stat_columns]
        away_team_data = avg_season_data.ix[tmp['VISITOR_TEAM_ID']][stat_columns]
        away_team_data = pd.DataFrame([list(away_team_data.values)], index = [game], columns = [x + '_AWAY' for x in list(away_team_data.index)])
        home_team_data = pd.DataFrame([list(home_team_data.values)], index = [game], columns = [x + '_HOME' for x in list(home_team_data.index)])
        data = pd.concat([home_team_data,away_team_data],1)
        X = X.append(data)
        
        if rows['WL'].ix[rows['TEAM_ID'] == tmp['HOME_TEAM_ID']].values[0] == 'W':
            Y.append(1)
        else:
            Y.append(0)


    return X,Y



In [12]:



def TreeRegressor(X, y):
    forest = ExtraTreesRegressor()
    clf = forest.fit(X, y)
    clf.feature_importances_
    model = SelectFromModel(clf, prefit=True)
    X_new = model.transform(X)





In [67]:
X,y = create_X(2)
X = X.drop(['GP_HOME', 'GP_AWAY'])


Number 0 out of 1230
Number 1 out of 1230
Number 2 out of 1230
Number 3 out of 1230
Number 4 out of 1230
Number 5 out of 1230
Number 6 out of 1230


KeyboardInterrupt: 

In [70]:
X.columns

Index([u'AST_HOME', u'BLK_HOME', u'BLKA_HOME', u'DREB_HOME', u'FG3A_HOME',
       u'FG3M_HOME', u'FG3_PCT_HOME', u'FGA_HOME', u'FGM_HOME', u'FG_PCT_HOME',
       u'FTA_HOME', u'FTM_HOME', u'FT_PCT_HOME', u'GP_HOME', u'L_HOME',
       u'MIN_HOME', u'OREB_HOME', u'PF_HOME', u'PFD_HOME', u'PLUS_MINUS_HOME',
       u'PTS_HOME', u'REB_HOME', u'STL_HOME', u'TOV_HOME', u'W_HOME',
       u'W_PCT_HOME', u'AST_AWAY', u'BLK_AWAY', u'BLKA_AWAY', u'DREB_AWAY',
       u'FG3A_AWAY', u'FG3M_AWAY', u'FG3_PCT_AWAY', u'FGA_AWAY', u'FGM_AWAY',
       u'FG_PCT_AWAY', u'FTA_AWAY', u'FTM_AWAY', u'FT_PCT_AWAY', u'GP_AWAY',
       u'L_AWAY', u'MIN_AWAY', u'OREB_AWAY', u'PF_AWAY', u'PFD_AWAY',
       u'PLUS_MINUS_AWAY', u'PTS_AWAY', u'REB_AWAY', u'STL_AWAY', u'TOV_AWAY',
       u'W_AWAY', u'W_PCT_AWAY'],
      dtype='object')

In [74]:
X.head()
X_norm = (X - X.mean()) / (X.max() - X.min())
X_norm

,AST_HOME,BLK_HOME,BLKA_HOME,DREB_HOME,FG3A_HOME,FG3M_HOME,FG3_PCT_HOME,FGA_HOME,FGM_HOME,FG_PCT_HOME,...,OREB_AWAY,PF_AWAY,PFD_AWAY,PLUS_MINUS_AWAY,PTS_AWAY,REB_AWAY,STL_AWAY,TOV_AWAY,W_AWAY,W_PCT_AWAY
0021401193,-0.141935,-2.309469e-02,2.309469e-02,-0.303779,-0.237185,-0.334081,-0.279739,0.015740,-0.365348,-0.3775,...,-0.330033,0.114780,-0.114727,-0.055034,-0.055034,-4.010363e-01,0.305031,-2.777778e-01,0.063830,0.064514
0021400534,0.139355,2.609700e-01,-2.609700e-01,0.135174,0.152042,0.219731,0.200654,-0.206716,0.263653,0.4225,...,-0.533003,0.073899,-0.073847,-0.346309,-0.346309,-6.766839e-01,-0.562893,1.984127e-01,-0.297872,-0.298487
0021400867,-0.490323,-3.256351e-01,3.256351e-01,0.127907,-0.311034,-0.459641,-0.456209,0.033578,-0.145009,-0.1775,...,-0.165017,-0.141509,0.141562,0.424832,0.424832,8.911917e-02,-0.075472,1.137566e-01,0.446809,0.446713
0021400866,-0.006452,-1.659116e-17,1.659116e-17,0.075581,0.204170,0.217489,0.073203,-0.046170,-0.171375,-0.1275,...,0.095710,0.040881,-0.040828,-0.394631,-0.394631,-1.088083e-01,-0.147799,4.391534e-01,-0.340426,-0.340372
0021400531,0.113548,-3.695150e-01,3.695150e-01,-0.106105,-0.054735,-0.141256,-0.220915,0.133263,-0.248588,-0.3400,...,0.003300,0.077044,-0.076992,0.144966,0.144966,1.761658e-02,0.235849,-1.526451e-17,0.148936,0.148284
0021401151,0.029677,2.078522e-01,-2.078522e-01,-0.236919,0.101651,0.103139,0.014379,0.354669,0.175141,-0.1150,...,-0.044554,0.248428,-0.248375,-0.575168,-0.575168,-3.544041e-01,0.091195,-1.851852e-02,-0.468085,-0.467772
0021401152,-0.490323,-3.256351e-01,3.256351e-01,0.127907,-0.311034,-0.459641,-0.456209,0.033578,-0.145009,-0.1775,...,0.095710,0.040881,-0.040828,-0.394631,-0.394631,-1.088083e-01,-0.147799,4.391534e-01,-0.340426,-0.340372
0021401153,-0.136774,1.362587e-01,-1.362587e-01,-0.479651,0.164205,0.242152,0.200654,-0.388248,-0.566855,-0.2275,...,-0.252475,-0.238994,0.239046,0.383893,0.383893,-6.217617e-02,0.418239,-3.306878e-01,0.340426,0.340256
0021401154,0.180645,-6.466513e-02,6.466513e-02,-0.178779,0.280626,0.255605,-0.015033,-0.162644,-0.192090,-0.0650,...,0.003300,0.077044,-0.076992,0.144966,0.144966,1.761658e-02,0.235849,-1.526451e-17,0.148936,0.148284
0021400537,0.295484,1.916859e-01,-1.916859e-01,0.238372,0.492615,0.529148,0.171242,-0.072403,-0.077213,-0.0150,...,-0.427393,-0.124214,0.124266,0.261745,0.261745,-2.922280e-01,0.204403,-4.153439e-01,0.276596,0.277429


In [83]:
forest = ExtraTreesRegressor()
clf = forest.fit(X_norm.ix[1:500], y[1:500])
clf.feature_importances_
model = SelectFromModel(clf, prefit=True)
feature_new = model.get_support(True)


clf = forest.fit(X_norm.ix[1:500][feature_new], y[1:500])
score = clf.score(X_norm.ix[500:][feature_new], y[500:])



In [84]:
score

-0.19548078654236067

In [33]:
feature_new

array([ 0,  2,  4,  5,  6, 14, 15, 16, 18, 21, 26, 29, 30, 34, 35, 37, 38,
       40, 41, 42, 43, 48, 50, 51])

In [30]:
X.columns[feature_new]

Index([u'AST_HOME', u'DREB_HOME', u'FG3A_HOME', u'FG3M_HOME', u'FG3_PCT_HOME',
       u'FGM_HOME', u'FTM_HOME', u'REB_HOME', u'STL_HOME', u'TOV_HOME',
       u'W_HOME', u'AST_AWAY', u'DREB_AWAY', u'FG3A_AWAY', u'FG3M_AWAY',
       u'FG3_PCT_AWAY', u'FGA_AWAY', u'FG_PCT_AWAY', u'FT_PCT_AWAY', u'L_AWAY',
       u'MIN_AWAY', u'OREB_AWAY', u'REB_AWAY', u'STL_AWAY', u'TOV_AWAY',
       u'W_AWAY', u'W_PCT_AWAY'],
      dtype='object')

In [37]:
clf.score(X_norm[feature_new], y[501:])

ValueError: Found arrays with inconsistent numbers of samples: [ 729 1230]

In [ ]:
X[feature_new]